In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

In [2]:
from modules.gm_lekerdezes_kezelo import *
from modules.osszakepacsolo import *

In [3]:
import geopandas as gpd, random
import leafmap.foliumap as leafmap
import matplotlib.pyplot as plt

from shapely.geometry import LineString
from shapely.ops import split
from shapely.ops import unary_union

In [4]:
# feldolgozva importálom a lekérdezett pontokat
df = read_jsonl_to_df('../adatok/fix/tokol_teljes_ms.jsonl')

3786 lekérdezett cím beolvasva


In [5]:
df

,geoid,cim,telepules,lat,lon
0,47289911900891,Csalitos u. 6,Tököl,47.289930,19.008894
1,47290591901043,Diana stny. 55,Tököl,47.290574,19.010528
2,47289711900922,Diana stny. 71,Tököl,47.289752,19.009213
3,47290281900996,Diana stny. 61,Tököl,47.290261,19.009986
4,47283961900063,Diana stny. 147,Tököl,47.283869,19.000709
...,...,...,...,...,...
3781,47311371897132,Fundamenta Személyibankár és Ingatlaniroda Tököl,Tököl,47.311245,18.971363
3782,47301881897500,None,Tököl,47.318706,18.967522
3783,47307351897590,Május 1. u. 6,Tököl,47.307348,18.975882
3784,47321091897891,Akácos út 50,Tököl,47.321084,18.978874


In [6]:
df['cim'].notna().sum()

np.int64(2367)

In [7]:
# a pontokat szűröm városra
df_t = filter_df_varos(df, "Tököl, Hungary")

In [6]:
df_t = cim_feldolgozas(df_t)

In [7]:
df_join = osszekapcs(df_t, None)

In [8]:
df_join

,geoid,szavazokorid,telepulesnev,utca,hazszam,lon,lat
0,47339011896848,1600934,Tököl,Duna u.,1,18.968487,47.339041
1,47340311896482,1600934,Tököl,Duna u.,10,18.964904,47.340311
2,47340881896416,1600934,Tököl,Duna u.,12,18.964142,47.340873
3,47339451896715,1600934,Tököl,Duna u.,3,18.967251,47.339439
4,47338911896761,1600934,Tököl,Duna u.,4,18.967634,47.338939
...,...,...,...,...,...,...,...
1782,47308531897548,1606585,Tököl,Deák Ferenc u.,5,18.975518,47.308484
1783,47308211897576,1606585,Tököl,Deák Ferenc u.,6,18.975731,47.308235
1784,47308401897534,1606585,Tököl,Deák Ferenc u.,7,18.975343,47.308407
1785,47308051897553,1606585,Tököl,Deák Ferenc u.,8,18.975520,47.308044


In [33]:
# 1. átalkítom a df_join-t gdf-re
gdf = gpd.GeoDataFrame(
    df_join,
    geometry=gpd.points_from_xy(df_join["lon"], df_join["lat"]),
    crs="EPSG:4326",
)

# 2. hozzárendelek külön színt a szavazókörökhöz

# Egyedi szavazókör ID-k
ids = gdf["szavazokorid"].unique()

# Színpaletta
cmap = plt.get_cmap("tab20")  # jó, kontrasztos színek
color_map = {
    sid: f"#{int(cmap(i)[0]*255):02x}{int(cmap(i)[1]*255):02x}{int(cmap(i)[2]*255):02x}"
    for i, sid in enumerate(ids)
}

# Szín oszlop a GDF-ben
gdf["color"] = gdf["szavazokorid"].map(color_map)

In [34]:
gdf

,geoid,szavazokorid,telepulesnev,utca,hazszam,lon,lat,geometry,color
0,47339011896848,1600934,Tököl,Duna u.,1,18.968487,47.339041,POINT (18.96849 47.33904),#1f77b4
1,47340311896482,1600934,Tököl,Duna u.,10,18.964904,47.340311,POINT (18.9649 47.34031),#1f77b4
2,47340881896416,1600934,Tököl,Duna u.,12,18.964142,47.340873,POINT (18.96414 47.34087),#1f77b4
3,47339451896715,1600934,Tököl,Duna u.,3,18.967251,47.339439,POINT (18.96725 47.33944),#1f77b4
4,47338911896761,1600934,Tököl,Duna u.,4,18.967634,47.338939,POINT (18.96763 47.33894),#1f77b4
...,...,...,...,...,...,...,...,...,...
1782,47308531897548,1606585,Tököl,Deák Ferenc u.,5,18.975518,47.308484,POINT (18.97552 47.30848),#ff9896
1783,47308211897576,1606585,Tököl,Deák Ferenc u.,6,18.975731,47.308235,POINT (18.97573 47.30823),#ff9896
1784,47308401897534,1606585,Tököl,Deák Ferenc u.,7,18.975343,47.308407,POINT (18.97534 47.30841),#ff9896
1785,47308051897553,1606585,Tököl,Deák Ferenc u.,8,18.975520,47.308044,POINT (18.97552 47.30804),#ff9896


In [43]:
gdf.to_file("../adatok/working/tokol_cimek_szinek.gpkg", driver="GPKG")

In [45]:
gdf_szigetek = gpd.read_file("../adatok/working/tokol_szigetek_v2.gpkg")

In [40]:
gdf

,geoid,szavazokorid,telepulesnev,utca,hazszam,lon,lat,geometry,color
0,47339011896848,1600934,Tököl,Duna u.,1,18.968487,47.339041,POINT (18.96849 47.33904),#1f77b4
1,47340311896482,1600934,Tököl,Duna u.,10,18.964904,47.340311,POINT (18.9649 47.34031),#1f77b4
2,47340881896416,1600934,Tököl,Duna u.,12,18.964142,47.340873,POINT (18.96414 47.34087),#1f77b4
3,47339451896715,1600934,Tököl,Duna u.,3,18.967251,47.339439,POINT (18.96725 47.33944),#1f77b4
4,47338911896761,1600934,Tököl,Duna u.,4,18.967634,47.338939,POINT (18.96763 47.33894),#1f77b4
...,...,...,...,...,...,...,...,...,...
1782,47308531897548,1606585,Tököl,Deák Ferenc u.,5,18.975518,47.308484,POINT (18.97552 47.30848),#ff9896
1783,47308211897576,1606585,Tököl,Deák Ferenc u.,6,18.975731,47.308235,POINT (18.97573 47.30823),#ff9896
1784,47308401897534,1606585,Tököl,Deák Ferenc u.,7,18.975343,47.308407,POINT (18.97534 47.30841),#ff9896
1785,47308051897553,1606585,Tököl,Deák Ferenc u.,8,18.975520,47.308044,POINT (18.97552 47.30804),#ff9896


In [46]:
gdf_szigetek

,geometry
0,"POLYGON ((345326.106 5242011.371, 345316.112 5..."
1,"POLYGON ((345288.365 5241969.184, 345293.3 524..."
2,"POLYGON ((345364.207 5242068.097, 345360.714 5..."
3,"POLYGON ((345478.637 5242021.59, 345494.939 52..."
4,"POLYGON ((345804.48 5242585.878, 345803.962 52..."
...,...
110,"POLYGON ((346937.481 5242447.346, 346938.208 5..."
111,"POLYGON ((347025.889 5242557.763, 347034.865 5..."
112,"POLYGON ((347125.839 5241304.669, 347172.638 5..."
113,"POLYGON ((347203.018 5242875.238, 347261.731 5..."


In [54]:
def list_points_inside_each_polygon(gdf, gdf_szigetek):
    """
    Végigmegy minden poligonon (gdf_szigetek), megkeresi a gdf pontokat, amik a poligonon belül vannak

    Visszaad egy listát, amiben poligononként eltesszük:
      - polygon_index / polygon_id
      - polygon_geometry
      - points_gdf (az összes megtalált pont soraival)
    """

    # CRS egységesítés: ugyanabban a vetületben kell lenniük a térbeli műveletekhez
    if gdf.crs != gdf_szigetek.crs:
        gdf = gdf.to_crs(gdf_szigetek.crs)

    # Végigmegyünk az összes poligonon
    
    for poly_idx, poly_row in gdf_szigetek.iterrows():
        polygon_geom = poly_row.geometry

        # Megkeressük a pontokat, amik "belül" vannak
        # contains -> szigorúan belül (a határvonalon lévő pont False)
        
        inside_mask = gdf.within(polygon_geom)
        points_inside = gdf[inside_mask].copy()

        print(points_inside)
        

        # 5) Kiírás: poligon + benne lévő pontok címei
        if len(points_inside) == 0:
            print(f"\nPoligon {poly_idx}: NINCS benne pont.")
        else:
            print(f"\nPoligon {poly_idx}: {len(points_inside)} pont található benne.")

In [55]:
list_points_inside_each_polygon(gdf, gdf_szigetek)

              geoid  szavazokorid telepulesnev         utca hazszam  \
534  47316241895590       1605395        Tököl  Ráckevei út      17   
535  47316311895577       1605395        Tököl  Ráckevei út      17   
536  47316281895564       1605395        Tököl  Ráckevei út      19   
537  47316141895576       1605395        Tököl  Ráckevei út      19   
539  47316021895568       1605395        Tököl  Ráckevei út      21   
540  47315891895558       1605395        Tököl  Ráckevei út      23   
541  47315791895545       1605395        Tököl  Ráckevei út      25   
542  47315851895533       1605395        Tököl  Ráckevei út      25   
543  47315661895533       1605395        Tököl  Ráckevei út      27   
544  47315531895517       1605395        Tököl  Ráckevei út      29   
545  47316021895568       1605395        Tököl  Ráckevei út      21   
546  47317301895655       1605395        Tököl  Ráckevei út       3   
547  47317201895677       1605395        Tököl  Ráckevei út       3   
548  4

In [94]:
def list_points_inside_each_polygon(gdf, gdf_szigetek):
    """
    Végigmegy minden poligonon, megkeresi a pontokat, és:
      - ha több szavazókör van egy poligonon belül -> print és skip
      - ha egyetlen szavazókör van -> results (GeoDataFrame) sorba menti:
          geometry (poligon), szavazokorid, color
    """

    # Biztonsági ellenőrzés
    if gdf.crs is None or gdf_szigetek.crs is None:
        raise ValueError("Mindkét GeoDataFrame-nek kell legyen CRS-e")

    # CRS egységesítés
    if gdf.crs != gdf_szigetek.crs:
        gdf = gdf.to_crs(gdf_szigetek.crs)

    # Ebbe gyűjtjük a "jó" poligonokat (amiknél 1 db szavazókör azonosítható)
    rows = []

    # Végigmegyünk az összes poligonon
    
    for poly_idx, poly_row in gdf_szigetek.iterrows():
        polygon_geom = poly_row.geometry

        # Pontok a poligonon belül
        inside_mask = gdf.within(polygon_geom)
        points_inside = gdf[inside_mask].copy()

        # Ha nincs pont, csak jelezzük és megyünk tovább
        if len(points_inside) == 0:
            #print(f"\nPoligon {poly_idx}: NINCS benne pont.")
            rows.append({"szavazokorid":None, "color":None, "geometry":polygon_geom})
            continue

        #print(f"\nPoligon {poly_idx}: {len(points_inside)} pont található benne.")

        
        # Egyedi szavazókörök a poligonon belül
        unique_szavazokorok = points_inside["szavazokorid"].dropna().unique()

        if len(unique_szavazokorok) != 1:
            #print("több szavazókörhöz tartozik")

            # meghívom a poly-n a rekúriv függvényt
            rows_darabok = polygon_tobb_szavazokor(polygon_geom, points_inside)
            rows.extend(rows_darabok)
            continue

        # Ha ide jutunk, akkor pontosan 1 szavazókör van
        szavazokorid_value = unique_szavazokorok[0]

        # Color: azonos (a szavazókörhöz)
        color_value = points_inside.iloc[0]["color"]

        # Mentjük a poligont a hozzárendelt szavazokorid-val és colorral
        rows.append({
            "szavazokorid": szavazokorid_value,
            "color": color_value,
            "geometry": polygon_geom
        })

    # Results GeoDataFrame
    results = gpd.GeoDataFrame(rows, geometry="geometry", crs=gdf_szigetek.crs)

    return results

In [95]:
# segéd függvények 

def felez(poly):
        """
        A poligont kettévágja a centroidon átmenő vágással (a hosszabb bbox tengely mentén).
        """
    
        minx, miny, maxx, maxy = poly.bounds
        cx, cy = poly.centroid.x, poly.centroid.y

        # Hosszabb irány kiválasztása
        if (maxx - minx) >= (maxy - miny):
            # függőleges vágás (x = cx)
            vago = LineString([(cx, miny - 1), (cx, maxy + 1)])
        else:
            # vízszintes vágás (y = cy)
            vago = LineString([(minx - 1, cy), (maxx + 1, cy)])

        darabok = list(split(poly, vago).geoms)

        # Ha valamiért nem sikerült a vágás akkor visszaadjuk egyben
        return darabok if len(darabok) >= 2 else [poly]

def pontok_poligonban(pts_gdf, poly):
    """
    Pontok szűrése poligonra
    """
    return pts_gdf[pts_gdf.within(poly)].copy()

def szavazokorok_szama(pts_gdf):
    """
    Hány különböző szavazokorid van a pontok között?
    """
    return pts_gdf["szavazokorid"].dropna().unique()

In [88]:
def polygon_tobb_szavazokor(polygon_geom, points_inside, max_depth=25):
    """
    Több szavazókörös poligon "szétszedése" felezéssel.

    Paraméterek:
      - polygon_geom: a poligon geometriája (shapely Polygon)
      - points_inside: GeoDataFrame, a poligonon belüli pontok (gdf szűrt része)

    Működés: 
    - Egy feldolgozási sorban (queue) tartjuk azokat a poligonokat, amik még kevertek
    - Minden körben: poligon felezése a mértani közepén
    - A felekre újraszűrjük a pontokat:
        - 0 pont -> üres poligon, nem bontjuk tovább
        - 1 db szavazokorid -> nem bontjuk tovább, megvan a legkisebb egyedi poly
        - több szavazokorid -> visszakerül a sorba, és újra felezzük
    
    """

    rows = []
    queue = [(polygon_geom, points_inside, 0)]  # (poly, pts, depth)

    # ameddig van nem egységes besorolású poligon
    while queue:
        poly, pts, depth = queue.pop() # kiveszünk egy polyt

        # ne legyen végtelen ciklus: ha túl mélyre mentünk inkább hadjuk
        if depth >= max_depth:
            print('Elérte a poly a mélységi szintet!')
            #rows.append({"szavazokorid":None, "color":None, "geometry":poly})
            continue

        # 1) felezés
        darabok = felez(poly)

        # 2) gyerekpoligonok értékelése
        for darab in darabok:
            darab_pts = pontok_poligonban(pts, darab) # lekérdezem a darabban lévő pontokat

            # ha 0 pont van benne
            if len(darab_pts) == 0:
                rows.append({"szavazokorid":None, "color":None, "geometry":darab})
                continue

            # megnézem hogy egyediek e szkid-k
            uniq = szavazokorok_szama(darab_pts)

            # csak 1 szavazókör -> eredmény ezt kell!!!
            if len(uniq) == 1:
                # mentem a polyt
                rows.append({
                    "szavazokorid": uniq[0],
                    "color": darab_pts.iloc[0]["color"],
                    "geometry": darab
                })
                continue

            # több szavazókör -> vissza a sorba, újra felezésre
            queue.append((darab, darab_pts, depth+1))

    return rows

In [96]:
results = list_points_inside_each_polygon(gdf, gdf_szigetek)

In [97]:
results

,szavazokorid,color,geometry
0,1605395.0,#ffbb78,"POLYGON ((345326.106 5242011.371, 345316.112 5..."
1,NaN,None,"POLYGON ((345288.365 5241969.184, 345293.3 524..."
2,1605395.0,#ffbb78,"POLYGON ((345364.207 5242068.097, 345360.714 5..."
3,1605395.0,#ffbb78,"POLYGON ((345478.637 5242021.59, 345494.939 52..."
4,1605395.0,#ffbb78,"POLYGON ((345804.48 5242585.878, 345803.962 52..."
...,...,...,...
264,1605796.0,#2ca02c,"POLYGON ((346937.481 5242447.346, 346938.208 5..."
265,1605796.0,#2ca02c,"POLYGON ((347025.889 5242557.763, 347034.865 5..."
266,NaN,None,"POLYGON ((347125.839 5241304.669, 347172.638 5..."
267,1605796.0,#2ca02c,"POLYGON ((347203.018 5242875.238, 347261.731 5..."


In [98]:
def fill_none_szavazokorid_by_neighbor_majority(results):
    """
    Azokat a sorokat kezeli, ahol szavazokorid hiányzik (NaN/None):
      - megkeresi a szomszédos poligonokat (touches: közös határ/pont érintés)
      - a szomszédok szavazokorid-jai közül a leggyakoribbat választja
      - beírja a hiányzó szavazokorid-t és a hozzá tartozó color-t (a nyertes szomszéd első colorja)

    Megjegyzés:
      - a "szomszéd" itt: geometriailag érintkező poligon (touches)
      - döntetlen esetén: a leggyakoribbak közül az első (deterministikus sorrend szerint) kerül kiválasztásra
    """

    out = results.copy()

    # Spatial index gyorsításhoz (olvasható marad, de nem lassú)
    sindex = out.sindex

    # Hiányzó szavazokorid sorok indexei (NaN is ide esik)
    missing_idxs = out.index[out["szavazokorid"].isna()].tolist()

    for idx in missing_idxs:
        geom = out.at[idx, "geometry"]

        # Jelöltek: bbox alapján (sindex), majd pontos szűrés touches-szal
        candidate_idxs = list(sindex.intersection(geom.bounds))
        candidates = out.loc[candidate_idxs]

        neighbors = candidates[candidates.geometry.touches(geom)]

        # Csak azok a szomszédok kellenek, ahol van szavazokorid
        neighbors_labeled = neighbors[neighbors["szavazokorid"].notna()]

        if len(neighbors_labeled) == 0:
            # nincs kitől örökölni -> marad NaN/None
            continue

        # Szavazokorid többség meghatározása
        counts = neighbors_labeled["szavazokorid"].value_counts()

        winner_szavazokorid = counts.index[0]

        # Color átvétele: az első olyan szomszédból, amelyik a nyertes szavazokorid
        winner_color = neighbors_labeled.loc[
            neighbors_labeled["szavazokorid"] == winner_szavazokorid, "color"
        ].iloc[0]

        out.at[idx, "szavazokorid"] = winner_szavazokorid
        out.at[idx, "color"] = winner_color

    return out

In [99]:
results_filled = fill_none_szavazokorid_by_neighbor_majority(results)

In [130]:
import geopandas as gpd
from shapely.ops import unary_union

def force_single_polygon_per_szavazokorid(
    results,
    *,
    max_parts: int = 1,
    start_tol: float = 0.1,
    grow_factor: float = 2.0,
    max_tol: float = 50.0,
):
    """
    Szavazókörönként egyetlen *Polygon*-t kényszerít ki úgy, hogy a különálló részeket
    toleranciás "ragasztással" összeköti (buffer+/-).

    - start_tol: kezdő ragasztási távolság (CRS egységben, EOV -> méter)
    - grow_factor: ha még mindig több part, ennyivel szorozzuk a tol-t
    - max_tol: biztonsági plafon, nehogy elszálljon

    FIGYELEM: ez torzít (hidakat képez), de cserébe 1 Polygon lesz.
    """

    out_rows = []

    for szkid, grp in results.groupby("szavazokorid", dropna=False):
        color = grp["color"].iloc[0] if "color" in grp.columns else None

        geom = unary_union(list(grp.geometry))
        tol = start_tol

        # addig "ragasztunk", amíg el nem érjük a kívánt parts számot (1)
        while True:
            if geom.geom_type == "Polygon":
                break

            if geom.geom_type == "MultiPolygon":
                parts = len(geom.geoms)
                if parts <= max_parts:
                    break
            else:
                # ha valami más (ritka), kilépünk
                break

            if tol > max_tol:
                # nem sikerült 1 poligonná kényszeríteni a plafonon belül
                break

            # closing: növeszt -> összeragad -> visszahúz
            geom = geom.buffer(tol).buffer(-tol)
            tol *= grow_factor

        # Ha még mindig MultiPolygon, itt dönthetsz: hagyod MultiPolygonként (1 geometria),
        # vagy kényszeríted burkolóval (convex hull). Most: visszaadjuk, ami lett.
        out_rows.append({
            "szavazokorid": szkid,
            "color": color,
            "geometry": geom
        })

    return gpd.GeoDataFrame(out_rows, geometry="geometry", crs=results.crs)


In [131]:
merged = force_single_polygon_per_szavazokorid(results_filled, start_tol=0.2, max_tol=20)

In [133]:
merged

,szavazokorid,color,geometry
0,1600934.0,#1f77b4,"MULTIPOLYGON (((346351.676 5244805.669, 346351..."
1,1603782.0,#aec7e8,"POLYGON ((346287.421 5244682.863, 346287.27 52..."
2,1604807.0,#ff7f0e,"POLYGON ((345844.109 5242867.296, 345844.111 5..."
3,1605395.0,#ffbb78,"MULTIPOLYGON (((345234.428 5241999.824, 345234..."
4,1605796.0,#2ca02c,"POLYGON ((345992.073 5242850.092, 345992.075 5..."
5,1606113.0,#98df8a,"MULTIPOLYGON (((346030.877 5242392.731, 346030..."
6,1606368.0,#d62728,"POLYGON ((345602.73 5241522.605, 345602.734 52..."
7,1606585.0,#ff9896,"MULTIPOLYGON (((346586.877 5241424.268, 346586..."


In [142]:
from shapely.ops import unary_union, polygonize, linemerge
from shapely.geometry import LineString, MultiLineString, GeometryCollection

def _chaikin_smooth_line(line: LineString, iterations: int = 2) -> LineString:
    coords = list(line.coords)
    if len(coords) < 3 or iterations <= 0:
        return line

    for _ in range(iterations):
        new_coords = [coords[0]]
        for i in range(len(coords) - 1):
            x0, y0 = coords[i]
            x1, y1 = coords[i + 1]
            q = (0.75 * x0 + 0.25 * x1, 0.75 * y0 + 0.25 * y1)
            r = (0.25 * x0 + 0.75 * x1, 0.25 * y0 + 0.75 * y1)
            new_coords.extend([q, r])
        new_coords.append(coords[-1])
        coords = new_coords

    return LineString(coords)

def _shared_to_lines(shared):
    """shared -> list[LineString] (linemerge csak akkor, ha kell és lehet)"""
    if shared.is_empty:
        return []

    if isinstance(shared, LineString):
        return [shared]

    if isinstance(shared, MultiLineString):
        merged = linemerge(shared)
        if isinstance(merged, LineString):
            return [merged]
        if isinstance(merged, MultiLineString):
            return list(merged.geoms)
        return []

    if isinstance(shared, GeometryCollection):
        lines = []
        for g in shared.geoms:
            if isinstance(g, LineString):
                lines.append(g)
            elif isinstance(g, MultiLineString):
                lines.extend(list(g.geoms))

        if not lines:
            return []

        merged = linemerge(unary_union(lines))
        if isinstance(merged, LineString):
            return [merged]
        if isinstance(merged, MultiLineString):
            return list(merged.geoms)
        return []

    return []

def straighten_border_between_two_polygons(poly_a, poly_b, *, simplify_tol=1.0, smooth_iters=2):
    """
    Két poligon közös (recés) határát lecseréli egy egyszerűbb + (opcionálisan) simított vonalra.
    Külső élekhez nem nyúl.

    Paraméterek (CRS egységben; EOV -> méter):
      - simplify_tol: recék eldobása (pl. 0.5 / 1.0 / 2.0)
      - smooth_iters: 0 = csak simplify; 1-3 = finom görbítés (Chaikin)

    Vissza:
      - (new_a, new_b)
    """

    # 1) közös határ
    shared = poly_a.boundary.intersection(poly_b.boundary)
    shared_lines = _shared_to_lines(shared)
    if not shared_lines:
        return poly_a, poly_b

    shared_u = unary_union(shared_lines)

    # 2) külső részek (közös nélkül)
    a_only = poly_a.boundary.difference(shared_u)
    b_only = poly_b.boundary.difference(shared_u)

    # 3) új közös határ: simplify + smoothing
    new_shared_lines = []
    for ln in shared_lines:
        ln2 = ln.simplify(simplify_tol)
        if smooth_iters > 0:
            ln2 = _chaikin_smooth_line(ln2, iterations=smooth_iters)
        new_shared_lines.append(ln2)

    new_shared = unary_union(new_shared_lines)

    # 4) új polygonok építése: külső élek + új közös él
    lines = unary_union([a_only, b_only, new_shared])
    pieces = list(polygonize(lines))
    if not pieces:
        return poly_a, poly_b

    # 5) darabok visszaosztása A/B oldalra
    a_parts, b_parts = [], []

    for p in pieces:
        rp = p.representative_point()
        in_a = rp.within(poly_a)
        in_b = rp.within(poly_b)

        if in_a and not in_b:
            a_parts.append(p)
        elif in_b and not in_a:
            b_parts.append(p)
        else:
            if p.intersection(poly_a).area >= p.intersection(poly_b).area:
                a_parts.append(p)
            else:
                b_parts.append(p)

    new_a = unary_union(a_parts) if a_parts else poly_a
    new_b = unary_union(b_parts) if b_parts else poly_b

    return new_a, new_b



def straighten_borders_gdf(merged, *, simplify_tol=1.0, smooth_iters=2):
    out = merged.copy()
    sidx = out.sindex
    done = set()

    for i, row_i in out.iterrows():
        gi = row_i.geometry
        for j in sidx.intersection(gi.bounds):
            if j == i:
                continue

            a, b = (i, j) if i < j else (j, i)
            if (a, b) in done:
                continue

            # csak eltérő szavazókör
            if out.at[i, "szavazokorid"] == out.at[j, "szavazokorid"]:
                continue

            gj = out.at[j, "geometry"]

            # csak ha tényleg összeérnek (közös határ)
            if not gi.touches(gj):
                continue

            new_i, new_j = straighten_border_between_two_polygons(
                gi, gj,
                simplify_tol=simplify_tol,
                smooth_iters=smooth_iters
            )

            out.at[i, "geometry"] = new_i
            out.at[j, "geometry"] = new_j
            done.add((a, b))

    return out


In [143]:
simitott = straighten_borders_gdf(merged, simplify_tol=1.0, smooth_iters=2)

In [144]:
simitott.to_file('../adatok/working/tokol_szigetek_besorolt_simitott.gpkg', layer='network_polygons', driver='GPKG')